In [1]:
!pip install webdriver-manager
!pip3 install lxml
!pip3 install selenium
!pip3 install webdriver_manager
!pip install --upgrade pip
!pip install -U selenium


  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/b5/b5/3bd0b038d80950ec13e6a2c8d03ed8354867dc60064b172f2f4ffac8afbe/webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/c9/33/b9da8be5b122b8c3c82c35f515ba0a84a9af3ba9629ae9fd5bbba820d592/selenium-4.23.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/9c/d6/7b05049918e5cb8cb2f16fa31078e8ddd3e062d5826e535d9bc2c2195b34/trio-0.26.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions~=4.9 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e012

In [8]:
import time 
from time import sleep 
import pandas as pd 
import selenium 
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException


In [9]:
# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications": 2}
chrome_options.add_experimental_option("prefs", prefs)

# Launch Chrome driver
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://ca.indeed.com/')
time.sleep(0.5)
driver.maximize_window()
time.sleep(15)


# Define function to send location input
def send_location_input(driver, location):
    WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, "text-input-where")))
    inputElement = driver.find_element(By.ID, "text-input-where")
    time.sleep(0.5)
    inputElement.send_keys(Keys.BACKSPACE * 16)
    time.sleep(0.5)
    inputElement.send_keys(location)
    inputElement.send_keys(Keys.ENTER)
    time.sleep(8)
    
# Define function to filter job postings by date
def filter_jobs_by_date(driver):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-controls='filter-dateposted-menu']"))).click()
    time.sleep(2)
    driver.find_element(By.XPATH, "//ul[@id='filter-dateposted-menu']/li[2]").click()
    time.sleep(8)

# Define function to close pop-up dialog
def close_popup_dialog(driver):
    try:
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//div[@role='dialog']//button[@aria-label='close']"))).click()
    except (NoSuchElementException, TimeoutException, StaleElementReferenceException):
        print("No pop-up dialog to close.")

# Define function to sort job listings by date
def sort_jobs_by_date(driver):
    driver.find_element(By.ID, 'filter-dateposted').click()
    time.sleep(6)
    driver.find_element(By.LINK_TEXT, 'date').click()

# Define function to extract job information
def extract_job_info(jobinfo):
    jobsite = []
    datepost1 = []
    titles = []
    company1 = []
    location1 = []
    pay1 = []
    emptype1 = []
    datepulled = []

    for job in jobinfo:
        jobsite.append("Indeed")

        title = job.find_elements(By.XPATH, ".//h2")
        titles.append(title[0].text if title else "None")

        datepost = job.find_element(By.XPATH, ".//span[@class='date']")
        datepost_text = datepost.text.lower().replace("posted", "").replace("\n", "").lstrip()
        datepost1.append(datepost_text if datepost_text else "None")

        company = job.find_elements(By.XPATH, ".//div//span[@class='companyName']")
        company1.append(company[0].text if company else "None")

        location = job.find_elements(By.XPATH, ".//div[@class='companyLocation']")
        location1.append(location[0].text if location else "None")

        pay = job.find_elements(By.XPATH, ".//div[@class='metadata salary-snippet-container']")
        pay1.append(pay[0].text if pay else "None")

        emptype = job.find_elements(By.XPATH, ".//div[@class='metadata'][1]")
        emptype1.append(emptype[0].text if emptype else "None")

        now = datetime.now()
        datepulled.append(now.strftime("%d/%m/%Y %H:%M:%S"))

    return jobsite, datepost1, titles, company1, location1, pay1, emptype1, datepulled

# Function to save data to Excel
def save_to_excel(data):
    df = pd.DataFrame(data)
    df.to_excel("IndeedJobs.xlsx", sheet_name="IndeedJobs", index=False)

# Main script execution
try:
    send_location_input(driver, "Northwest Territories")
    filter_jobs_by_date(driver)
    close_popup_dialog(driver)
    sort_jobs_by_date(driver)

    jobsite, datepost1, titles, company1, location1, pay1, emptype1, datepulled = [], [], [], [], [], [], [], []

    while True:
        time.sleep(3)
        jobinfo = driver.find_elements(By.XPATH, "//ul[@class='jobsearch-ResultsList css-0']//div[contains(@class, 'cardOutline')]")
        print(len(jobinfo))

        job_data = extract_job_info(jobinfo)
        jobsite.extend(job_data[0])
        datepost1.extend(job_data[1])
        titles.extend(job_data[2])
        company1.extend(job_data[3])
        location1.extend(job_data[4])
        pay1.extend(job_data[5])
        emptype1.extend(job_data[6])
        datepulled.extend(job_data[7])

        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@data-testid='pagination-page-next']"))).click()
        except (NoSuchElementException, TimeoutException):
            break

    data = {'jobsite': jobsite, 'datepost1': datepost1, 'titles': titles, 'company1': company1, 'location1': location1, 'pay1': pay1, 'emptype1': emptype1, 'datepulled': datepulled}
    save_to_excel(data)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    driver.close()

An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.89)
Stacktrace:
0   chromedriver                        0x0000000109819838 chromedriver + 5179448
1   chromedriver                        0x000000010981126a chromedriver + 5145194
2   chromedriver                        0x00000001093882b0 chromedriver + 389808
3   chromedriver                        0x000000010935f614 chromedriver + 222740
4   chromedriver                        0x00000001093fe8ed chromedriver + 874733
5   chromedriver                        0x00000001094134a9 chromedriver + 959657
6   chromedriver                        0x00000001093f6553 chromedriver + 841043
7   chromedriver                        0x00000001093c77f6 chromedriver + 649206
8   chromedriver                        0x00000001093c805e chromedriver + 651358
9   chromedriver                        0x00000001097dcaa0 chromedriver + 4930208
10  chromedriver        

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.89)
Stacktrace:
0   chromedriver                        0x0000000109819838 chromedriver + 5179448
1   chromedriver                        0x000000010981126a chromedriver + 5145194
2   chromedriver                        0x00000001093882b0 chromedriver + 389808
3   chromedriver                        0x000000010935f614 chromedriver + 222740
4   chromedriver                        0x00000001093fe8ed chromedriver + 874733
5   chromedriver                        0x0000000109405123 chromedriver + 901411
6   chromedriver                        0x00000001093f6778 chromedriver + 841592
7   chromedriver                        0x00000001093c77f6 chromedriver + 649206
8   chromedriver                        0x00000001093c805e chromedriver + 651358
9   chromedriver                        0x00000001097dcaa0 chromedriver + 4930208
10  chromedriver    